In [1]:
import random
from greedy import greedy_cit
from testsuite import TestSuite
from ga import GA

In [2]:
a = (greedy_cit(2, ["a1", "a2", "a3", "a4", "a5"], [10, 10, 10, 10, 10]))                 


In [3]:
b = (greedy_cit(3, ["a1", "a2", "a3", "a4", "a5"], [10, 10, 10, 10, 10]))     
                                              

In [4]:
temp = GA(b, a, 5, 10, 900, 800, 0.7)


In [5]:
temp.delete_base_from_greedy()

In [8]:
temp.initial_population()

In [ ]:
for i in range(100):
    print(i)
    temp.fitness_calculate([],[])
    temp.next_generation()
    temp.adjust()
